[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/notebooks/blob/main/ap's_360_panorama.ipynb)

In [ ]:
#step 1
!pip install diffusers==0.19.1 transformers accelerate streamlit streamlit_pannellum

In [ ]:
#step2
from diffusers import StableDiffusionLDM3DPipeline
from PIL import Image
from typing import Optional
from torch import Tensor
from torch.nn import functional as F
from torch.nn import Conv2d
from torch.nn.modules.utils import _pair

# Load the pipeline
pipe = StableDiffusionLDM3DPipeline.from_pretrained("Intel/ldm3d-pano")
pipe.to("cuda")

# Function to override _conv_forward method
def asymmetricConv2DConvForward(self, input: Tensor, weight: Tensor, bias: Optional[Tensor]):
    paddingX = (self._reversed_padding_repeated_twice[0], self._reversed_padding_repeated_twice[1], 0, 0)
    paddingY = (0, 0, self._reversed_padding_repeated_twice[2], self._reversed_padding_repeated_twice[3])
    working = F.pad(input, paddingX, mode='circular')
    working = F.pad(working, paddingY, mode='constant')
    return F.conv2d(working, weight, bias, self.stride, _pair(0), self.dilation, self.groups)

# Patch the Conv2d layers
targets = [pipe.vae, pipe.text_encoder, pipe.unet]
for target in targets:
    for module in target.modules():
        if isinstance(module, Conv2d):
            module._conv_forward = asymmetricConv2DConvForward.__get__(module, Conv2d)

In [ ]:
# Define the prompt and filename base
prompt = "The living room"
name = "jungle_pano"

# Generate the images
output = pipe(prompt, width=1024, height=512, guidance_scale=7.0, num_inference_steps=50)

# Extract the RGB and depth images
rgb_image, depth_image = output.rgb, output.depth

# Save images to files
rgb_image[0].save(name+"_ldm3d_rgb.jpg")
depth_image[0].save(name+"_ldd3d_depth.png")

# Display images
rgb_image[0].show()
depth_image[0].show()

#step3
from PIL import Image
from IPython.display import display

# File names from the saved images
rgb_filename = name + "_ldm3d_rgb.jpg"

# Open the saved images
rgb_saved_image = Image.open(rgb_filename)

# Display the images
display(rgb_saved_image)

In [ ]:
# upscale step 1
!pip install -q realesrgan==0.3.0
!wget https://huggingface.co/camenduru/sd-t2i-360panoimage/resolve/main/RealESRGAN_x2plus.pth?download=true -O /content/RealESRGAN_x2plus.pth

In [ ]:
# upscale step 2
from PIL import Image
rgb_saved_image = Image.open('/content/jungle_pano_ldm3d_rgb.jpg')

from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

sr_model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32,scale=2)
upsampler = RealESRGANer(scale=2, model_path='/content/RealESRGAN_x2plus.pth', dni_weight=None, model=sr_model, tile=384, tile_pad=20, pre_pad=20, half=False, device='cuda',)

import numpy as np
output_img = rgb_saved_image.resize((1024 * 2, 512 * 2))
w = output_img.size[0]
blend_extend = 10
outscale = 2
output_img = np.array(output_img)
output_img = np.concatenate([output_img, output_img[:, :blend_extend, :]], axis=1)
output_img, _ = upsampler.enhance(output_img, outscale=outscale)
output_img = Image.fromarray(output_img[:, :w * outscale, :])
output_img